In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import statistics
matplotlib.rcParams['axes.unicode_minus'] = False
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [107]:
get_data = pd.read_csv('../data/TEMP/경락가격/애호박/애호박 월별 총거래(정제).csv', index_col=None)

In [108]:
get_data['Date'] = pd.to_datetime(get_data['Date'])

In [109]:
years = get_data['Date'].dt.year.unique()

In [110]:

year_mean = []
year_addon = []
for i in years:
    for j in get_data.columns:
        if '평균' in j:
            data_p = get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values
            year_addon.append(data_p.sum())
            # np_std= np.std(get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values)
            # print('표준편차: ',np_std)
            # year_addon.append(np_std)
        elif j != 'Date':
            data_p = get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values
            year_addon.append(data_p.sum())
        else:
            year_addon.append(i)
    year_mean.append(year_addon)
    year_addon = []

year_mean

[[2020,
  51102052.3,
  101783384400,
  25988.072122596248,
  5660056.1,
  12674733776,
  28730.080370511805,
  13161830.0,
  21677981900,
  20721.621926051484,
  6090057.0,
  11588035170,
  23919.399775180762,
  6830758.7,
  13988202166,
  26100.967508096306],
 [2021,
  46174142.760000005,
  96540979170,
  27237.50759138543,
  5753657.0,
  13221287030,
  30181.800179689482,
  13593558.0,
  23273513218,
  22090.96828743961,
  4506057.0,
  9951150666,
  27385.399831836196,
  6168655.0,
  13356367665,
  27534.954951566524],
 [2022,
  47791699.419999994,
  108330801715,
  28723.828948027316,
  6384089.899999999,
  15242087730,
  30174.079700780665,
  13555823.0,
  24664233900,
  22822.441079591576,
  4866515.0,
  11134510089,
  28820.686019361136,
  7026107.5,
  15231159196,
  28014.850853381242],
 [2023,
  28206576.29,
  68388891702,
  20784.42409442935,
  3980311.4000000004,
  10402027600,
  22297.424840606884,
  7653050.0,
  16120285400,
  17020.948742481232,
  3000827.0,
  7650796664,

In [111]:
year_df = pd.DataFrame(year_mean, columns=['year', '서울 총거래물량', '서울 총거래금액', '서울 평균단가','부산 총거래물량', '부산 총거래금액', '부산 평균단가', '대구 총거래물량', '대구 총거래금액', '대구 평균단가', '대전 총거래물량', '대전 총거래금액', '대전 평균단가', '광주 총거래물량', '광주 총거래금액', '광주 평균단가'])

In [112]:
for i in years:
    year_df.loc[year_df['year']== i,'서울 평균단가'] = year_df['서울 총거래금액']/year_df['서울 총거래물량']
    year_df.loc[year_df['year']== i,'부산 평균단가'] = year_df['부산 총거래금액']/year_df['부산 총거래물량']
    year_df.loc[year_df['year']== i,'대구 평균단가'] = year_df['대구 총거래금액']/year_df['대구 총거래물량']
    year_df.loc[year_df['year']== i,'대전 평균단가'] = year_df['대전 총거래금액']/year_df['대전 총거래물량']
    year_df.loc[year_df['year']== i,'광주 평균단가'] = year_df['광주 총거래금액']/year_df['광주 총거래물량']

year_df

,year,서울 총거래물량,서울 총거래금액,서울 평균단가,부산 총거래물량,부산 총거래금액,부산 평균단가,대구 총거래물량,대구 총거래금액,대구 평균단가,대전 총거래물량,대전 총거래금액,대전 평균단가,광주 총거래물량,광주 총거래금액,광주 평균단가
0,2020,51102052.30,101783384400,1991.767059,5660056.1,12674733776,2239.330062,13161830.0,21677981900,1647.034029,6090057.0,11588035170,1902.779427,6830758.7,13988202166,2047.825546
1,2021,46174142.76,96540979170,2090.801765,5753657.0,13221287030,2297.892806,13593558.0,23273513218,1712.098717,4506057.0,9951150666,2208.394316,6168655.0,13356367665,2165.199329
2,2022,47791699.42,108330801715,2266.728387,6384089.9,15242087730,2387.511449,13555823.0,24664233900,1819.456768,4866515.0,11134510089,2287.984336,7026107.5,15231159196,2167.794785
3,2023,28206576.29,68388891702,2424.572589,3980311.4,10402027600,2613.370301,7653050.0,16120285400,2106.387048,3000827.0,7650796664,2549.562725,4523155.0,11418522574,2524.459713


In [113]:
get_data = get_data.rename(columns={'서울 평균거래금액':'서울 평균단가', '부산 평균거래금액':'부산 평균단가', '대구 평균거래금액':'대구 평균단가', '대전 평균거래금액':'대전 평균단가', '광주 평균거래금액':'광주 평균단가'})

In [114]:
year_df['전국 평균단가'] = (year_df['서울 총거래금액'] + year_df['부산 총거래금액'] + year_df['대구 총거래금액'] + year_df['대전 총거래금액'] + year_df['광주 총거래금액']) / (year_df['서울 총거래물량'] + year_df['부산 총거래물량'] + year_df['대구 총거래물량'] + year_df['대전 총거래물량'] + year_df['광주 총거래물량']) 

In [115]:
years = get_data['Date'].dt.year.unique()
months = get_data['Date'].dt.month.unique()
grade_data = []
for i in years:
    for j in months:
        if ( i != 2023 ) or ( j not in [9, 10, 11, 12]):
            g_date = str(i)+"-"+str(j)
            seoul_data = year_df.loc[year_df['year']==i, '서울 평균단가'].values
            seoul_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '서울 평균단가'].values
            seoul_grade = seoul_data - seoul_data1

            busan_data = year_df.loc[year_df['year']==i, '부산 평균단가'].values
            busan_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '부산 평균단가'].values
            busan_grade = busan_data - busan_data1

            daegu_data = year_df.loc[year_df['year']==i, '대구 평균단가'].values
            daegu_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '대구 평균단가'].values
            daegu_grade = daegu_data - daegu_data1

            daejeon_data = year_df.loc[year_df['year']==i, '대전 평균단가'].values
            daejeon_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '광주 평균단가'].values
            daejeon_grade = daejeon_data - daejeon_data1

            gwangju_data = year_df.loc[year_df['year']==i, '광주 평균단가'].values
            gwangju_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '광주 평균단가'].values
            gwangju_grade = gwangju_data - gwangju_data1
            grade_data.append([g_date,seoul_grade[0], busan_grade[0], daegu_grade[0], daejeon_grade[0], gwangju_grade[0]])

In [116]:
mean_data = pd.DataFrame(grade_data, columns=['Date','서울 평균단가 편차', '부산 평균단가 편차', '대구 평균단가 편차', '대전 평균단가 편차', '광주 평균단가 편차'])

In [117]:
mean_data['Date'] = pd.to_datetime(mean_data['Date'])

In [118]:
mean_all = pd.merge(get_data, mean_data, on='Date', how='inner')

In [119]:
mean_all = mean_all[['Date', '서울 총거래물량', '서울 총거래금액', '서울 평균단가', '서울 평균단가 편차','부산 총거래물량', '부산 총거래금액', '부산 평균단가', '부산 평균단가 편차', '대구 총거래물량', '대구 총거래금액', '대구 평균단가', '대구 평균단가 편차','대전 총거래물량', '대전 총거래금액', '대전 평균단가', '대전 평균단가 편차', '광주 총거래물량', '광주 총거래금액', '광주 평균단가', '광주 평균단가 편차']]

In [127]:
mean_all['전국 평균단가'] = (mean_all['서울 총거래금액'] + mean_all['부산 총거래금액'] + mean_all['대구 총거래금액'] + mean_all['대전 총거래금액'] + mean_all['광주 총거래금액']) / (mean_all['서울 총거래물량'] + mean_all['부산 총거래물량'] + mean_all['대구 총거래물량'] + mean_all['대전 총거래물량'] + mean_all['광주 총거래물량']) 

In [ ]:
years = mean_all['Date'].dt.year.unique()
months = mean_all['Date'].dt.month.unique()
all_grade_data = []
for i in years:
    for j in months:
        if ( i != 2023 ) or ( j not in [9, 10, 11, 12]):
            g_date = str(i)+"-"+str(j)
            total_data = year_df.loc[year_df['year']==i, '전국 평균단가'].values
            total_data1 = mean_all.loc[(mean_all['Date'].dt.month == j) & (mean_all['Date'].dt.year==i), '전국 평균단가'].values
            total_grade = total_data - total_data1

            all_grade_data.append([g_date,total_grade[0]])
print(all_grade_data)

[['2020-1', -1372.6737629937745], ['2020-2', -1175.1719396070373], ['2020-3', -544.7245211032612], ['2020-4', 565.8781132858703], ['2020-5', 684.6457574607098], ['2020-6', 693.3455775554817], ['2020-7', 409.49700732280917], ['2020-8', -950.6002993963118], ['2020-9', -1036.5615568230926], ['2020-10', 765.6285753700911], ['2020-11', 511.0563856349777], ['2020-12', -272.5275730058381], ['2021-1', -1533.9131244154933], ['2021-2', -1094.5544814068453], ['2021-3', -548.2440373511304], ['2021-4', 468.24703055487976], ['2021-5', 396.7025621060691], ['2021-6', 554.8992293816393], ['2021-7', 1191.6749709683481], ['2021-8', 340.2475366069766], ['2021-9', -124.55052490598109], ['2021-10', -210.41202448836248], ['2021-11', -335.4279853787257], ['2021-12', -1005.8754079414216], ['2022-1', -1533.6236008737433], ['2022-2', -420.37497500251766], ['2022-3', -389.8466866325148], ['2022-4', 589.5557399828576], ['2022-5', 913.4551753061387], ['2022-6', 416.9073641720115], ['2022-7', 64.03933356797643], ['2

In [ ]:
total_df = pd.DataFrame(all_grade_data, columns=['Date', '전국 평균단가 편차'])

In [ ]:
total_df['Date'] = pd.to_datetime(total_df['Date'])

In [ ]:
total_all = pd.merge(mean_all, total_df, on='Date', how='inner')

In [121]:
total_all.to_csv('../data/TEMP/경락가격/애호박/애호박 월별 데이터와 평균편차(전국포함).csv', index=False, encoding='UTF-8')

In [122]:
get_data = pd.read_csv('../data/TEMP/경락가격/애호박/애호박 월별 데이터와 평균편차(전국포함).csv', index_col=None)

In [123]:
get_data

,Date,서울 총거래물량,서울 총거래금액,서울 평균단가,서울 평균단가 편차,부산 총거래물량,부산 총거래금액,부산 평균단가,부산 평균단가 편차,대구 총거래물량,...,대구 평균단가 편차,대전 총거래물량,대전 총거래금액,대전 평균단가,대전 평균단가 편차,광주 총거래물량,광주 총거래금액,광주 평균단가,광주 평균단가 편차,전국 평균단가
0,2020-01-01,2780377.65,9830108400,3535.529931,-1543.762872,314022.5,1158346710,3688.737941,-1449.407879,810553.0,...,-860.657016,430426.0,1337955110,3108.443983,-1536.211640,478129.0,1644281360,3438.991067,-1391.165521,3324.666248
1,2020-02-01,2720669.88,9024401700,3316.977839,-1325.210781,331341.0,1143201466,3450.226401,-1210.896339,827334.0,...,-743.621860,406261.0,1228049490,3022.809204,-1286.961089,451781.0,1441064160,3189.740516,-1141.914970,3127.164424
2,2020-03-01,3739541.28,9722242800,2599.849038,-608.081980,435282.4,1220358529,2803.601820,-564.271759,978456.0,...,-330.683313,520843.0,1246237300,2392.731207,-666.689269,587348.0,1509172300,2569.468697,-521.643150,2496.717006
3,2020-04-01,5703989.95,7969557560,1397.189972,594.577086,607195.5,995696080,1639.827831,599.502230,1427859.0,...,465.590974,615618.0,884077957,1436.082046,461.396143,800667.0,1154068030,1441.383284,606.442262,1386.114372
4,2020-05-01,5487463.43,7107073600,1295.147328,696.619730,644394.2,875168986,1358.126727,881.203334,1371680.0,...,529.403022,631163.0,818889984,1297.430274,667.920800,759615.0,938017136,1234.858627,812.966919,1267.346727
5,2020-06-01,5435012.79,6912510040,1271.847980,719.919078,556802.2,827896497,1486.877202,752.452860,1417036.0,...,527.086124,623977.0,715574424,1146.796154,549.762902,715343.0,967870900,1353.016525,694.809021,1258.646907
6,2020-07-01,4369081.64,6969965000,1595.292918,396.474140,517931.6,935421697,1806.071877,433.258185,1249719.0,...,394.817251,554574.0,806098452,1453.545338,277.894227,545036.0,885620930,1624.885200,422.940346,1542.495478
7,2020-08-01,3399364.19,10140301900,2982.999565,-991.232506,419775.4,1354083040,3225.732237,-986.402175,1036770.0,...,-853.501094,401571.0,1112157900,2769.517470,-1134.063018,456631.2,1386717010,3036.842445,-989.016899,2902.592784
8,2020-09-01,4424172.16,13495784500,3050.465491,-1058.698432,425793.8,1461229576,3431.777485,-1192.447423,1069058.0,...,-944.127722,463269.0,1232979303,2661.475952,-1339.144621,448876.5,1455223520,3241.924048,-1194.098502,2988.554042
9,2020-10-01,5212849.56,6208943600,1191.084363,800.682695,509824.0,787313146,1544.284196,695.045866,1129309.0,...,640.683243,485244.0,526468105,1084.955414,633.744755,549347.0,697140390,1269.034672,778.790874,1186.363909


In [124]:
get_data['전국 총거래물량'] = get_data['서울 총거래물량'] + get_data['부산 총거래물량'] + get_data['대구 총거래물량'] + get_data['대전 총거래물량'] + get_data['광주 총거래물량']
get_data['전국 총거래금액'] = get_data['서울 총거래금액'] + get_data['부산 총거래금액'] + get_data['대구 총거래금액'] + get_data['대전 총거래금액'] + get_data['광주 총거래금액']

In [125]:
seoul_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('서울')])
seoul_data = get_data[seoul_columns]

busan_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('부산')])
busan_data = get_data[busan_columns]

daegu_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대구')])
daegu_data = get_data[daegu_columns]

daejeon_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대전')])
daejeon_data = get_data[daejeon_columns]

gwangju_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('광주')])
gwangju_data = get_data[gwangju_columns]

aways_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('전국')])
aways_data = get_data[aways_columns]

In [126]:
aways_data = aways_data[['Date', '전국 총거래물량', '전국 총거래금액', '전국 평균단가', '전국 평균단가 편차']]

KeyError: "['전국 평균단가 편차'] not in index"

In [ ]:
seoul_data = seoul_data.rename(columns={'서울 총거래물량':'총거래물량', '서울 총거래금액':'총거래금액', '서울 평균단가':'평균단가', '서울 평균단가 편차':'평균단가 편차'})
busan_data = busan_data.rename(columns={'부산 총거래물량':'총거래물량', '부산 총거래금액':'총거래금액', '부산 평균단가':'평균단가', '부산 평균단가 편차':'평균단가 편차'})
daegu_data = daegu_data.rename(columns={'대구 총거래물량':'총거래물량', '대구 총거래금액':'총거래금액', '대구 평균단가':'평균단가', '대구 평균단가 편차':'평균단가 편차'})
daejeon_data = daejeon_data.rename(columns={'대전 총거래물량':'총거래물량', '대전 총거래금액':'총거래금액', '대전 평균단가':'평균단가', '대전 평균단가 편차':'평균단가 편차'})
gwangju_data = gwangju_data.rename(columns={'광주 총거래물량':'총거래물량', '광주 총거래금액':'총거래금액', '광주 평균단가':'평균단가', '광주 평균단가 편차':'평균단가 편차'})
aways_data = aways_data.rename(columns={'전국 총거래물량':'총거래물량', '전국 총거래금액':'총거래금액', '전국 평균단가':'평균단가', '전국 평균단가 편차':'평균단가 편차'})

In [ ]:
seoul_data['지역'] = '서울'
busan_data['지역'] = '부산'
daegu_data['지역'] = '대구'
daejeon_data['지역'] = '대전'
gwangju_data['지역'] = '광주'
aways_data['지역'] = '전국'

In [ ]:
all_data = pd.concat([seoul_data, busan_data, daegu_data, daejeon_data, gwangju_data, aways_data])

In [ ]:
all_data.to_csv('../data/TEMP/경락가격/애호박/애호박 컬럼 통합한 데이터와 표준편차.csv', index=False, encoding='UTF-8')